In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

In [3]:
import numpy as np

In [5]:
#from PIL import Image
from skimage import io, transform

In [6]:
import h5py

# Convert matlab data to jpg image

In [ ]:
data_folder = "/home/user170/shared-data/Personal_Dev/Machine-Learning/Data/Depth/NYU-Depth-Dataset-V2/"

In [ ]:
rgb_img = Image.open(data_folder+"Images/RGB/0000.jpg")
d_img = Image.open(data_folder+"Images/Depth-TIFF/0000.tiff")

In [ ]:
plt.imshow(rgb_img)

In [ ]:
plt.imshow(d_img)
print(np.max(d_img))
print(np.min(d_img))

In [ ]:
import os

#sorted(os.listdir(data_folder+"Images/Depth-TIFF"))

In [ ]:
class LoadData(Dataset):
    def __init__(self, root_dir, testset_ratio=0.3, transform=None):
        """
        Args:
            root_dir (string): Directory with RGB and Depth-TIFF images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.mat_data = h5py.File(root_dir+mat_file, 'r')
        self.root_dir = root_dir
        self.transform = transform
        self.testset_ratio = testset_ratio
        
        self.trainset_idx, self.testset_idx, self.trainset_num, self.testset_num = self._seperate_dataset()

    def _seperate_dataset(self):
        """
        Seperate Train and Test dataset
        
        Returns: random indice of train test datasets, and lend of those.
        """
        dataset_len = len(os.listdir(self.root_dir+"Images/RGB/"))
        rnd_idx = [i for i in range(dataset_len)]
        np.random.shuffle(rnd_idx)
        split_thresh = int(dataset_len*(1-self.testset_ratio))

        train_idx = rnd_idx[:split_thresh]
        test_idx = rnd_idx[split_thresh:]

        trainset_num = len(train_idx)
        testset_num = len(test_idx)

        return train_idx, test_idx, trainset_num, testset_num
    
    def __len__(self):
        self.dataset_len = len(os.listdir(self.root_dir+"Images/RGB/"))
        return self.dataset_len
    
    def __getitem__(self, idx):
        rgb_img = Image.open(self.root_dir+'Images/RGB/{0:04d}.jpg'.format(self.trainset_idx[idx]))
        depth_img = Image.open(self.root_dir+'Images/Depth-TIFF/{0:04d}.tiff'.format(self.trainset_idx[idx]))
        
        sample = {'RGB':rgb_img, 'DEPTH':depth_img}
            
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [ ]:
trainloader = LoadData(data_folder)

In [ ]:
for i in range(len(trainloader)):
    sample = trainloader[i]
    print(i, 
          '[RGB u: {:.2f}, std: {:.2f}]'.format(np.mean(sample['RGB']), np.std(sample['RGB'])),
          '\t[DEPTH u: {:.2f}, std: {:.2f}]'.format(np.mean(sample['DEPTH']), np.std(sample['DEPTH'])))
    
    if i == 3:
        #plt.show()
        break

# Transformation

In [ ]:
class ToTensor(object):
    """Convert ndarray in sample to Tensors."""
    
    def __call__(self, sample):
        rgb_img, depth_img = sample['RGB'], sample['DEPTH']
        
        # swap colour axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        rgb_img = np.moveaxis(np.asarray(rgb_img), [0], [-0])
        depth_img = np.asarray(depth_img)
        
        return {'RGB': torch.from_numpy(rgb_img), 'DEPTH': torch.from_numpy(depth_img)}

In [ ]:
to_tensor = ToTensor()
plt.imshow(to_tensor(train[0])['RGB'].numpy())

In [ ]:
composed = transforms.Compose([ToTensor()])
print(type(composed(train[0])['RGB'])
plt.imshow(composed(train[0])['RGB'].numpy())

In [ ]:
plt.imshow(composed(train[0])['RGB'].numpy())

### Using class function

In [ ]:
load_data = LoadData(root_dir=data_folder, transform=composed)

ax = plt.subplot(2, 1, 1)
plt.imshow(load_data[0]['RGB'].numpy())
ax = plt.subplot(2, 1, 2)
plt.imshow(load_data[0]['DEPTH'].numpy())

# DataLoader

In [ ]:
np.max(load_data[0]['RGB'].numpy())

Make normalization class